In [1]:
import sys
import os
import glob
import string
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [2]:
#Data directory
data_dir = 'biflow-unlabeled'
filenames = glob.glob(data_dir+'/*.csv')
files_raw = list()
previous_row = list()
dfs = list()
#Check if the unlabeled files have the same number and kind of features
for file in filenames:
    df = pd.read_csv(file, index_col=None, header=0)
    dfs.append(df)
    with open(file, newline='') as f:
        reader = csv.reader(f)
        row = next(reader)
        if len(previous_row)==0:
            previous_row = row
            files_raw.append(file[len(data_dir)+1:len(file)-14])
            continue
        if previous_row!=row:
            print('Error: different features in different files')
            break
        files_raw.append(file[len(data_dir)+1:len(file)-14])
#All the features generated by CIC Flowmeter
allFeatures = previous_row

In [3]:
#Application name for each data file as inferred from data file name
files= ['skype_audio',
'facebook_chat',
'email',
'sftp_up',
'skype_chat',
'vimeo',
'hangouts_chat',
'vimeo',
'ftps_down',
'facebook_audio',
'netflix',
'aim_chat',
'spotify',
'icq_chat',
'facebook_chat',
'Vimeo',
'skype_files',
'Twitter',
'email',
'vimeo',
'hangout_chat',
'youtube',
'email',
'youtube',
'ftps_down',
'hangouts_audio',
'netflix',
'facebook_video',
'scpDown',
'scpDown',
'skype_file',
'spotify',
'vimeo',
'Google',
'netflix',
'hangouts_chat',
'skype_chat',
'scpDown',
'Youtube',
'voipbuster',
'skype_video',
'youtube',
'youtube',
'facebook_audio',
'hangouts_audio',
'bittorrent',
'hangouts_audio',
'skype_file',
'skype_audio',
'vimeo',
'email',
'scpDown',
'facebook_audio',
'ftps_up',
'scpUp',
'scpDown',
'voipbuster',
'Youtube',
'netflix',
'icq_chat',
'voipbuster',
'skype_video',
'spotify',
'sftp_down',
'facebook_chat',
'skype_file',
'hangouts_chat',
'aim_chat',
'voipbuster',
'spotify',
'icq_chat',
'sftpDown',
'sftpDown',
'scpUp',
'Facebook',
'skype_audio',
'Youtube',
'youtube',
'skype_audio',
'sftp_down',
'youtube',
'vimeo',
'ftps',
'spotify',
'youtube',
'email',
'sftp',
'ftps',
'skype_files',
'skype_chat',
'hangouts_video',
'icq_chat',
'sftp',
'sftpUp',
'scpUp',
'ftps_up',
'hangouts_audio',
'aim_chat',
'scpUp',
'facebook_video',
'email',
'youtube',
'sftp_up',
'facebook_chat',
'netflix',
'skype_chat',
'aim_chat',
'scpDown',
'scpUp',
'Vimeo',
'Vimeo']

In [4]:
#Adding Application name to Label column of dataframes and concatinating all dattaframes
for df,f in zip(dfs,files):
    df.loc[:,'Label'] = f.upper()
frame = pd.concat(dfs, axis=0, ignore_index=True)
frame.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,204.236.238.164-10.8.8.138-21-50940-6,10.8.8.138,50940,204.236.238.164,21,6,22/05/2015 03:47:09 PM,229824,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
1,204.236.238.164-10.8.8.138-1731-42086-6,10.8.8.138,42086,204.236.238.164,1731,6,22/05/2015 03:47:24 PM,4327892,91,194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
2,204.236.238.164-10.8.8.138-1731-42086-6,10.8.8.138,42086,204.236.238.164,1731,6,22/05/2015 03:47:28 PM,296,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
3,204.236.238.164-10.8.8.138-1587-45798-6,10.8.8.138,45798,204.236.238.164,1587,6,22/05/2015 03:47:29 PM,3926182,142,217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
4,204.236.238.164-10.8.8.138-1587-45798-6,10.8.8.138,45798,204.236.238.164,1587,6,22/05/2015 03:47:33 PM,590,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO


In [5]:
#List of all features except:
#'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port','Timestamp'
features = ['Protocol', 'Flow Duration'\
, 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min'\
, 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean'\
, 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max'\
, 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot'\
, 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags'\
, 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s'\
, 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt'\
, 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt'\
, 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg'\
, 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts'\
, 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts'\
, 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max', 'Active Min'\
, 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label']

In [6]:
#Keep only above mentioned features
for af in allFeatures:
    if af not in features:
        frame = frame.drop(columns=af)
garbage = features.pop()

In [7]:
#Data cleaning
syn_flag = frame['SYN Flag Cnt'].values
cnt = 0
invalid_idx = list()
for a in syn_flag:
    try:
        val = float(a)
    except:
        #print(a)
        frame.loc[cnt,'SYN Flag Cnt'] = '0'
        invalid_idx.append(cnt)
    cnt=cnt+1
frame['Flow Pkts/s'] = frame['Flow Pkts/s'].astype(float)
frame['SYN Flag Cnt'] = frame['SYN Flag Cnt'].astype(float)
#Checking Nan
idx,col = np.where(np.isnan(frame[features]))
nan_idx = np.ndarray.tolist(idx)
#Removing rows containing Nan
frame = frame.drop(frame.index[nan_idx])
frame.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,229824,0,2,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
1,6,4327892,91,194,481.0,243832.0,283.0,0.0,5.285714,32.876295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
2,6,296,1,1,0.0,229.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
3,6,3926182,142,217,273.0,273735.0,107.0,0.0,1.922535,13.270280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO
4,6,590,2,1,85.0,0.0,85.0,0.0,42.500000,60.104076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SKYPE_AUDIO


In [8]:
# Random forest classifier on unnormalized statistical flow features data
# Set random seed
np.random.seed(0)
#Create training and test data
frame['is_train'] = np.random.uniform(0, 1, len(frame)) <= .75
#Class label to integer mapping for each observation
y,target_names = pd.factorize(frame['Label'])
frame = frame.assign(Label_id = y) 

# Create two new dataframes, one with the training rows, one with the test rows
train, test = frame[frame['is_train']==True], frame[frame['is_train']==False]

# number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 34760
Number of observations in the test data: 11542


In [9]:
# Create a random forest Classifier. clf means 'Classifier'
clf = RandomForestClassifier(n_estimators=1000, n_jobs=4, random_state=0)
# Train the Classifier
clf.fit(train[features], train['Label_id'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=4, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [10]:
# Print the name and gini importance of each feature
print('Gini importance of each feature:')
for feat in zip(features, clf.feature_importances_):
    print(feat)


('Protocol', 0.0017011143389129542)
('Flow Duration', 0.06482344964623848)
('Tot Fwd Pkts', 0.004428729041552803)
('Tot Bwd Pkts', 0.004213946825903176)
('TotLen Fwd Pkts', 0.004884705254057215)
('TotLen Bwd Pkts', 0.015002821259408933)
('Fwd Pkt Len Max', 0.005431174170701558)
('Fwd Pkt Len Min', 0.00447417417256984)
('Fwd Pkt Len Mean', 0.005077007634248087)
('Fwd Pkt Len Std', 0.0036574861338126733)
('Bwd Pkt Len Max', 0.01531757785007538)
('Bwd Pkt Len Min', 0.01153780099648886)
('Bwd Pkt Len Mean', 0.01454649930890845)
('Bwd Pkt Len Std', 0.016967838687611397)
('Flow Byts/s', 0.055077964117893695)
('Flow Pkts/s', 0.06241949519197948)
('Flow IAT Mean', 0.06317573842265047)
('Flow IAT Std', 0.011709718649234844)
('Flow IAT Max', 0.06379474427348437)
('Flow IAT Min', 0.0648444887304455)
('Fwd IAT Tot', 0.008698657808678754)
('Fwd IAT Mean', 0.008510521363469263)
('Fwd IAT Std', 0.005813135378592376)
('Fwd IAT Max', 0.008741991783668903)
('Fwd IAT Min', 0.00972338021791007)
('Bwd IAT 

In [11]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than Threshold
sfm = SelectFromModel(clf, threshold=0.05)

# Train the selector
sfm.fit(train[features], train['Label_id'])

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=1000, n_jobs=4,
                                                 oob_score=False,
                                                 random_state=0, verbose=0,
       

In [12]:
# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(features[feature_list_index])

Flow Duration
Flow Byts/s
Flow Pkts/s
Flow IAT Mean
Flow IAT Max
Flow IAT Min
Bwd Pkts/s


In [13]:
# Transform the data to create a new dataset containing only the most important features
X_important_train = sfm.transform(train[features])
X_important_test = sfm.transform(test[features])

In [14]:
# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=4)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, train['Label_id'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=4, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
# Apply The Full Featured Classifier To The Test Data
y_pred = clf.predict(test[features])
y_test = test['Label_id'].values
print('Accuracy Full Feature Classifier: ' + str(accuracy_score(y_test, y_pred)))

# Apply The Important Feature Classifier To The Test Data
y_pred = clf_important.predict(X_important_test)
y_test = test['Label_id'].values
print('Accuracy Important Feature Classifier: ' + str(accuracy_score(y_test, y_pred)))

Accuracy Full Feature Classifier: 0.5428002079362328
Accuracy Important Feature Classifier: 0.4871772656385375


In [17]:
# Create confusion matrix for full classifier
preds = target_names[clf.predict(test[features])]
pd.crosstab(test['Label'], preds, rownames=['Actual Class'], colnames=['Predicted Class'])

Predicted Class,AIM_CHAT,BITTORRENT,EMAIL,FACEBOOK,FACEBOOK_AUDIO,FACEBOOK_CHAT,FACEBOOK_VIDEO,FTPS,FTPS_DOWN,FTPS_UP,...,SKYPE_AUDIO,SKYPE_CHAT,SKYPE_FILE,SKYPE_FILES,SKYPE_VIDEO,SPOTIFY,TWITTER,VIMEO,VOIPBUSTER,YOUTUBE
Actual Class,,,,,,,,,,,,,,,,,,,,,
AIM_CHAT,333,0,13,0,2,0,3,0,23,18,...,2,3,4,2,1,5,4,11,20,17
BITTORRENT,0,25,2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
EMAIL,12,2,242,1,0,0,1,0,2,4,...,5,3,5,0,4,3,4,20,13,18
FACEBOOK,0,0,0,3,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
FACEBOOK_AUDIO,4,0,5,0,19,1,1,0,13,6,...,0,3,4,0,1,0,0,7,11,18
FACEBOOK_CHAT,0,0,1,0,4,30,0,0,0,4,...,1,0,0,0,3,1,0,3,5,5
FACEBOOK_VIDEO,0,0,0,0,1,0,13,0,4,0,...,1,0,1,0,0,0,0,2,4,1
FTPS,0,0,0,0,2,0,0,2,0,0,...,0,0,3,0,1,1,0,4,0,2
FTPS_DOWN,12,0,1,0,15,1,5,0,164,21,...,0,0,4,0,0,3,0,17,74,148


In [19]:
# Train Random forest classifier for features provided by ISCX
#Same features were used for coarse classification (7 classes)
features_small = ['Flow Duration', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max'\
, 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot'\
, 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Active Mean', 'Active Std', 'Active Max'\
, 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label','is_train','Label_id']

In [21]:
#Dropping extra features from dataframe
for af in features:
    if af not in features_small:
        frame = frame.drop(columns=af)
garbage = features_small.pop()
garbage = features_small.pop()
garbage = features_small.pop()

In [22]:
train, test = frame[frame['is_train']==True], frame[frame['is_train']==False]
# Create a random forest Classifier.
clf = RandomForestClassifier(n_estimators=1000, n_jobs=4, random_state=0)
# Train the Classifier
clf.fit(train[features_small], train['Label_id'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=4, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [23]:
#Accuracy score of classifier trained with same features used of coarse analysis
y_pred = clf.predict(test[features_small])
y_test = test['Label_id'].values
print('Accuracy with same features used of coarse analysis: ' + str(accuracy_score(y_test, y_pred)))

Accuracy with same features used of coarse analysis: 0.4979206376711142
